In [2]:
from stim import Circuit, Tableau, PauliString, TableauSimulator
import numpy as np

In [3]:
def get_stabilizers_from_tableau(s: TableauSimulator):
    return s.current_inverse_tableau().inverse().to_stabilizers()[:-1]

Unitary, $U$, stabilizes pure state, $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle, \ \alpha, \ \beta \in \mathbb{C}$, if $U|\psi\rangle = |\psi\rangle$; $U$ stabilizes the eigenstate of $U$ with $+1$ eigenvalue. Note that global phase bears great revelance here: $U$ does not stabilize $|\psi\rangle$ if $U|\psi\rangle = -|\psi\rangle$. The Pauli matrices, $P \in \{I, \ Y, \ X, \ Z\}$, stabilize
$$Z|0\rangle = |0\rangle, \ -Z|1\rangle = |1\rangle,$$
$$X|+\rangle = |+\rangle, \ -X|-\rangle = |-\rangle,$$
$$Y|i\rangle = |i\rangle, \ -Y|{-i}\rangle = |{-i}\rangle,$$
$I$ stabilizes any $|\psi\rangle$, and $-I$ stabilizes no $|\psi\rangle$.
Recall some important qualities of Pauli matrices:
$$iXY = Z, \ iYX = -Z,$$
$$iYZ = X, \ iZY = -X,$$
$$iZX = Y, \ iXZ = -Y,$$
Pauli matrices anti-commute with each other,
$$\{P_i, P_j\} = P_iP_j + P_jP_i = 0, \ i \neq j, $$
that all Pauli matrices are closed under multiplication,
$$P_i^2 = I,$$
Conveniently, the Pauli operators represent the errors we care about:
\
No error:
$$I|\psi\rangle = \alpha|0\rangle + \beta|1\rangle.$$
Bit-flip error:
$$X|\psi\rangle = \beta|0\rangle + \alpha|1\rangle.$$
Phase-flip error:
$$Z|\psi\rangle = \alpha|0\rangle -\beta|1\rangle.$$
Both phase- and bit-flip error:
$$iZX|\psi\rangle = Y|\psi\rangle = \beta|0\rangle - \alpha|1\rangle.$$
Stabilizers are also associative such that for another unitary, $V$, that also stabilizes $|\psi\rangle$, any product of $U$ and $V$ also stabilize $|\psi\rangle$. Now consider the case of an $2$ qubit Bell state, $|\Psi\rangle$, the stabilizers of $|\Psi\rangle$ can be found by a composition of tensor products that stabilize each state in $|\Psi\rangle$, making a stabilizer *group*, $S$. We know always $I\otimes I \otimes ... \otimes I \in S$, so we will omit it out of simplicity. For the sake of brevity, consider the notation $\{I\otimes I \otimes ... \otimes I\} \rightarrow \{II...I\}$; the stabilizer group for $|\Psi\rangle$ is 

### Repetition Code - Revisited
As previous, consider two logical codewords that make up the codespace, $\mathcal{C}$: 
$$|0\rangle_L \equiv |000\rangle, \ |1\rangle_L \equiv |111\rangle, \ |0\rangle_L, \ |1\rangle_L \in \mathcal{C}.$$
$|0\rangle_L$, $|1\rangle_L$ are orthogonal. The stabilizer generator group for this code is $S = \{ZZI, ZIZ\}$

In [4]:
bitflip_rep_generator_group = [
        PauliString('ZZ_'), # S_1
        PauliString('Z_Z'), # S_2
        PauliString('ZZZ'), 
    ]

bitflip_rep=TableauSimulator()
bitflip_rep.set_state_from_stabilizers(
    bitflip_rep_generator_group
)

In [5]:
bitflip_rep_x_err_2 = bitflip_rep.copy()
bitflip_rep_x_err_2.do_pauli_string(PauliString('_X_'))
get_stabilizers_from_tableau(bitflip_rep_x_err_2)

[stim.PauliString("-ZZ_"), stim.PauliString("+Z_Z")]

In [6]:
bitflip_rep_x_err_2 = bitflip_rep.copy()
bitflip_rep_x_err_2.do_pauli_string(PauliString('X__'))
get_stabilizers_from_tableau(bitflip_rep_x_err_2)

[stim.PauliString("-ZZ_"), stim.PauliString("-Z_Z")]

In [7]:
bitflip_rep_x_err_2 = bitflip_rep.copy()
bitflip_rep_x_err_2.do_pauli_string(PauliString('__X'))
get_stabilizers_from_tableau(bitflip_rep_x_err_2)

[stim.PauliString("+ZZ_"), stim.PauliString("-Z_Z")]

We can see that each error produces a distinct effect on the stabilizer group. This is important as it satisfies a key component of quantum error correction. 

$$

In [8]:
phaseflip_rep_generator_group = [
        PauliString('XX_'), # S_1
        PauliString('X_X'), # S_2
        PauliString('XXX'), 
    ]

phaseflip_rep=TableauSimulator()
phaseflip_rep.set_state_from_stabilizers(
    phaseflip_rep_generator_group
)


In [9]:
shors_generator_group = [
        PauliString('ZZ_______'), # S_1 
        PauliString('Z_Z______'), # S_2
        PauliString('___ZZ____'), # S_3
        PauliString('___Z_Z___'), # S_4
        PauliString('______ZZ_'), # S_5
        PauliString('______Z_Z'), # S_6
        PauliString('XXX___XXX'), # S_7
        PauliString('XXXXXX___'), # S_8, notice how all of the stabilizer generators commute!
        PauliString('XXXXXXXXX'), #+1 equiv. |0>_L, -1 equiv |1>_L
    ]

s = TableauSimulator()
s.set_state_from_stabilizers(
    shors_generator_group
)

bell_p = np.zeros((8,))
np.put(bell_p, [0, 7], 1/np.sqrt(2)) # (1 / sqrt(2))(|000> + |111>)
logical_0 = np.kron(np.kron(bell_p, bell_p), bell_p) # |0>_L = (1 / (2 * sqrt(2)))(|000> + |111>)(|000> + |111>)(|000> + |111>)

# Will be true if shors_generator_group correctly encodes the state as outlined in Shor's 
print(f'Encoded state from stabilizers match expected: {np.allclose(logical_0, s.state_vector())}')
get_stabilizers_from_tableau(s)

Encoded state from stabilizers match expected: True


[stim.PauliString("+ZZ_______"),
 stim.PauliString("+Z_Z______"),
 stim.PauliString("+___ZZ____"),
 stim.PauliString("+___Z_Z___"),
 stim.PauliString("+______ZZ_"),
 stim.PauliString("+______Z_Z"),
 stim.PauliString("+XXX___XXX"),
 stim.PauliString("+XXXXXX___")]

Suppose there is a single arbitrary Pauli error, $\{E\} \in P$, that occurs after encoding.

In [10]:
s_x_err = s.copy()
s_x_err.do_pauli_string(PauliString('__X______')) # X error on 3rd qubit
get_stabilizers_from_tableau(s_x_err)

[stim.PauliString("+ZZ_______"),
 stim.PauliString("-Z_Z______"),
 stim.PauliString("+___ZZ____"),
 stim.PauliString("+___Z_Z___"),
 stim.PauliString("+______ZZ_"),
 stim.PauliString("+______Z_Z"),
 stim.PauliString("+XXX___XXX"),
 stim.PauliString("+XXXXXX___")]

Now, the state is 
$${X_3}|0\rangle_L \rightarrow \frac{1}{2\sqrt{2}}\left((|000\rangle+|111\rangle)(|100\rangle+|011\rangle)(|000\rangle+|111\rangle)\right).$$
Note that the stabilizers in $S_2$ anti-commute while the rest of the generators commute. 

In [11]:
s_x_err.do_pauli_string(PauliString('__X______')) # Correct X error on 3rd qubit
get_stabilizers_from_tableau(s_x_err)

[stim.PauliString("+ZZ_______"),
 stim.PauliString("+Z_Z______"),
 stim.PauliString("+___ZZ____"),
 stim.PauliString("+___Z_Z___"),
 stim.PauliString("+______ZZ_"),
 stim.PauliString("+______Z_Z"),
 stim.PauliString("+XXX___XXX"),
 stim.PauliString("+XXXXXX___")]

And we are back to our original state.

In [12]:
s_z_err = s.copy()
s_z_err.do_pauli_string(PauliString('_______Z_')) # Z error on 8th qubit
get_stabilizers_from_tableau(s_z_err)

[stim.PauliString("+ZZ_______"),
 stim.PauliString("+Z_Z______"),
 stim.PauliString("+___ZZ____"),
 stim.PauliString("+___Z_Z___"),
 stim.PauliString("+______ZZ_"),
 stim.PauliString("+______Z_Z"),
 stim.PauliString("-XXX___XXX"),
 stim.PauliString("+XXXXXX___")]

Now, the state is 
$$ Z_8|0\rangle \rightarrow \frac{1}{2\sqrt{2}}\left((|000\rangle+|111\rangle)(|000\rangle+|111\rangle)(|000\rangle-|111\rangle)\right).$$
For phase flip errors, we see that a phase-flip affects the entire block! We cannot tell precisely which qubit caused the phase-flip (assuming we didn't know about the intentional error I applied to the state)... 

After applying 

We can find the logical observable by measuring the ${X}_L$ observable.

In [13]:
s_meas_0 = s.copy()
s_meas_0.peek_observable_expectation(PauliString('XXXXXXXXX'))     

1

We measure $|{0}\rangle_L$ (or $+1$ expectation value of ${X}_L$) as expected! To flip from $|{0}\rangle_L \rightarrow |{1}\rangle_L$, we can apply a ${Z}_L$ operator. We should now get $|{1}\rangle_L$ when we measure ${X}_L$.

In [14]:
s = TableauSimulator()
s.set_state_from_stabilizers(
    shors_generator_group
)
s.do_pauli_string(PauliString('ZZZZZZZZZ'))

s.peek_observable_expectation(PauliString('XXXXXXXXX'))   

-1

In [15]:
c=Tableau.from_stabilizers(shors_generator_group).to_circuit()
for i in range(9): c.append('DEPOLARIZE1', i, .1)
s = c.compile_sampler()